In [1]:
#%%
import numpy as np
import pdb

In [2]:
#%%initialisations
class Agent():
    def __init__(self):
        self.row = 5
        self.column = 5

        #states in the grid world
        self.state1 = np.empty((self.row,self.column))

        #value for each state
        self.state_value = {}
        value = 1
        for i in range(self.row):
            for j in range(self.column):
                self.state1[i,j] = value
                self.state_value[value] = None
                value +=1

        #action space
        self.action = {6: 'left', 7: 'right', 8: 'north', 9: 'south'}

    def action_state(self,action,state):
#        pdb.set_trace()
        reward = 0
        tot_rew = 0.0
        state1 = np.inf
        loop = True
        #leftmost corner condition
        if state in (self.state1[0,0],self.state1[4,0],\
                     self.state1[1,0],self.state1[2,0],\
                     self.state1[3,0]):
            if action == self.action[6]:
                reward = -1
                state1 = state
            if action == self.action[7]:
                reward = 0
                state1 = state + 1
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward  = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
            loop = False

        #right most condition
        if state == self.state1[0,4] or\
        state == self.state1[1,4] or \
        state == self.state1[2,4] or \
        state == self.state1[3,4] or \
        state == self.state1[4,4]:
            if action == self.action[6]:
                reward = 0
                state1 = state - 1
            if action == self.action[7]:
                reward = -1
                state1 = state
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
            loop = False


        #best conditions
        if state == self.state1[0,1]:
            action = self.action
            reward = 10
            state1 = state + 20
            loop = False
            tot_rew = reward

        if state  == self.state1[0,3]:
            action = self.action
            reward = 5
            state1 = state + 15
            tot_rew = reward
            loop = False

        elif loop == True:
            if action == self.action[6]:
                state1 = state - 1
                reward = 0
            if action == self.action[7]:
                state1 = state + 1
                reward = 0
            if action == self.action[8]:
                state1 = state - 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state - 5
            if action == self.action[9]:
                state1 = state + 5
                if state1 not in self.state1:
                    reward = -1
                    state1 = state
                else:
                    reward = 0
                    state1 = state + 5
            tot_rew += reward
        return state1, reward

In [3]:
#%%initialising policy evaluation
x_agent = Agent()
counter = 0
for i in range(x_agent.row):
    for j in range(x_agent.column):
        counter +=1
# Number of states.
num_states = counter
a= x_agent.row*x_agent.column

# Number of actions and possible actions.
action_space = x_agent.action.keys()
num_actions = len(action_space)

# Discount factor
gamma = 0.9


    
temp = 0
prob = np.asarray((x_agent.row,x_agent.column))
prob = np.zeros((prob))
state_transit = np.zeros((x_agent.row*x_agent.column,num_actions))
rew = np.zeros((x_agent.row*x_agent.column,num_actions))
update = 0
for r in range(x_agent.row):
    for c in range(x_agent.column):
        ac = np.arange(6,10)
        for l in range(len(ac)):
            [state_transit[update,l],rew[update,l]] =\
            x_agent.action_state(x_agent.action[ac[l]], x_agent.state1[r,c])
        update+=1
#finding the probability of being in a state        
prob1 = np.zeros((a,a)) #finding p(pi)
prob = np.zeros((a,a))  
     
for p in range(a):
    p_v = 1 
    for q in range(a):
        prob[p,q] = p_v
        p_v +=1
for row in range(a):
    for column in range(a):
        for k in range(num_actions):
            if prob[row,column] == state_transit[row,k]:
                prob1[row,column] += 0.25

#finding the reward
rew1 = np.zeros((a,1))
for i in range(a):
    rew1[i] = 0.25*np.sum(rew[i])

#finding value function
I = np.identity(25)
value = np.zeros((25))
m = np.linalg.inv(I - gamma*prob1)
n = np.matmul(m,rew1)

In [5]:
n.T

array([[ 3.2821087 ,  8.7623075 ,  4.09704936,  4.40306405,  0.99767746,
         1.48284711,  2.92163238,  2.05779847,  1.55769357,  0.25792529,
         0.01495451,  0.68207971,  0.56939569,  0.20429472, -0.55585057,
        -1.00230572, -0.47451719, -0.4135254 , -0.66326217, -1.2657053 ,
        -1.88171257, -1.37521389, -1.26950698, -1.47289588, -2.02942776]])